In [14]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
from tqdm import tqdm
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
import numpy as np
from sentence_transformers import SentenceTransformer,util
import matplotlib.pyplot as plt
import csv
from numpy.linalg import norm
from numpy import dot

/opt/anaconda3/envs/mysite/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib is building the font cache; this may take a moment.


In [15]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [16]:
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

In [17]:
sentences=["학식 메뉴 알려줘","편의시설에 대해서 알려줘","연락처 검색을 하고싶어","학사 일정이 궁금해","공지 사항이 궁금해","날씨 정보","홍익 대학교 기본 상식에 대해서 알려줘","홍익 대학교 건학 이념","졸업하기 위한 필요 조건에 대해서 알려줘","열람실 현황 정보"]

In [18]:
f=open('dataset/all_data.txt')

test_sentence=f.readlines()[:500]

embedding_vectors=[model.encode(sentence) for sentence in test_sentence]
embedding_vectors=torch.tensor(embedding_vectors)
print(embedding_vectors.shape)

torch.Size([500, 768])


/var/folders/6t/w80zhm5565j25gfph102_yfm0000gn/T/ipykernel_32434/1384670176.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:277.)
  embedding_vectors=torch.tensor(embedding_vectors)


In [19]:
def cos_sim(A, B):
    return dot(A, B) / (norm(A) * norm(B))

In [39]:
def predict(sentence):
    encoded_sentence=model.encode(sentence)
    encoded_sentence=torch.tensor(encoded_sentence)
    cos_sim=util.cos_sim(encoded_sentence,embedding_vectors)
    best_sim_idx=int(np.argmax(cos_sim))
    selected_question=test_sentence[best_sim_idx]
    if float(cos_sim.max()) < 0.45 :
        print(f"원래 문장 : {sentence}")
        print(f"가장 높은 코사인 유사도 idx: {int(np.argmax(cos_sim))}")
        print(f"선택된 질문 : {selected_question}")
        print(f"선태괸 질문과의 유사도 : {float(cos_sim.max())}")

    return int(np.argmax(cos_sim))

In [23]:
s = "홍익대학교 마스코트"

import time

start_time=time.time()

predict(s)

end_time=time.time()

print(end_time-start_time)

원래 문장 : 홍익대학교 마스코트
가장 높은 코사인 유사도 idx: 446
선택된 질문 : 갤러리아에 새로 생겼움...

선태괸 질문과의 유사도 : 0.3722449243068695
0.059454917907714844


In [25]:
data=pd.read_csv("dataset/my_data_0702.csv")

In [26]:
correct=0
total_cnt=len(data)
f=open("failure.txt",'w')
for i in tqdm(range(len(data))):
    label=data.iloc[i]['label']
    if label == 7 or label == 8:
        total_cnt-=1
        continue
    res=predict(data.iloc[i]['sentence'])
    if res==data.iloc[i]['label']-1:
        correct+=1
    else:
        f.write(f"원래 문장 : {data.iloc[i]['sentence']}, 원래 라벨 : {data.iloc[i]['label']-1}, 예측 라벨 : {res}\n")

  0%|          | 0/4487 [00:00<?, ?it/s]

원래 문장 : 오늘 홍대 날씨 좋아요?


  9%|▉         | 401/4487 [00:00<00:02, 1653.23it/s]

가장 높은 코사인 유사도 idx: 192
선택된 질문 : 음 신선한데?

선태괸 질문과의 유사도 : 0.405082643032074
원래 문장 : 오늘 홍대 날씨 맑아?
가장 높은 코사인 유사도 idx: 157
선택된 질문 : 난 물이 무서워서 겁나

선태괸 질문과의 유사도 : 0.4562638998031616
원래 문장 : 홍대 날씨 지금 맑은가요?
가장 높은 코사인 유사도 idx: 157
선택된 질문 : 난 물이 무서워서 겁나

선태괸 질문과의 유사도 : 0.4280480146408081
원래 문장 : 홍대 오늘 날씨 화창한 거 같아?
가장 높은 코사인 유사도 idx: 288
선택된 질문 : 응 그래서 한참 난리 났었는데 어디 백화점이였지 ㅠㅠ

선태괸 질문과의 유사도 : 0.46110329031944275
원래 문장 : 홍익대학교 날씨가 화사한지 궁금해요.
가장 높은 코사인 유사도 idx: 370
선택된 질문 : 지성인분들 겨울에 부러워요

선태괸 질문과의 유사도 : 0.501961350440979
원래 문장 : 홍대 날씨 지금 쾌청해?
가장 높은 코사인 유사도 idx: 157
선택된 질문 : 난 물이 무서워서 겁나

선태괸 질문과의 유사도 : 0.4539240002632141
원래 문장 : 홍대 오늘 날씨가 쾌청한가요?
가장 높은 코사인 유사도 idx: 370
선택된 질문 : 지성인분들 겨울에 부러워요

선태괸 질문과의 유사도 : 0.4354976415634155
원래 문장 : 홍익대학교 오늘 날씨 청명한가요?
가장 높은 코사인 유사도 idx: 370
선택된 질문 : 지성인분들 겨울에 부러워요

선태괸 질문과의 유사도 : 0.41625040769577026
원래 문장 : 오늘 홍익대학교 날씨 맑니?
가장 높은 코사인 유사도 idx: 157
선택된 질문 : 난 물이 무서워서 겁나

선태괸 질문과의 유사도 : 0.3938139081001282
원래 문장 : 홍대 날씨 오늘 맑아요?
가장 높은 코사인 유사도 idx: 157
선택된 질문 : 난 물이 무서워서 

 13%|█▎        | 567/4487 [00:03<00:29, 131.90it/s] 

...

가장 높은 코사인 유사도 idx: 38
선택된 질문 : 예약을 했는데 숙박업소에서 오류라고 전화 오더라구요

선태괸 질문과의 유사도 : 0.3322056829929352
원래 문장 : 전자공학과의 최신 공지사항은 무엇인가요?
가장 높은 코사인 유사도 idx: 267
선택된 질문 : 좋아하는 팀 있어?

선태괸 질문과의 유사도 : 0.3087182641029358
원래 문장 : 건축학과의 최근 업데이트된 공지사항은 어디서 볼 수 있나요?
가장 높은 코사인 유사도 idx: 369
선택된 질문 : 전 워낙 지성이라 잘 안바르게되더라고요

선태괸 질문과의 유사도 : 0.3549204468727112
원래 문장 : 화학공학과의 오늘 올라온 공지사항이 있나요?
가장 높은 코사인 유사도 idx: 267
선택된 질문 : 좋아하는 팀 있어?

선태괸 질문과의 유사도 : 0.35041505098342896
원래 문장 : 산업디자인학과 공지사항 페이지의 URL은 무엇인가요?
가장 높은 코사인 유사도 idx: 310
선택된 질문 : 사귀면 인증좀 부탁해도 될까?

선태괸 질문과의 유사도 : 0.29167813062667847
원래 문장 : 생명과학과 공지사항 검색 방법을 알려주세요.
가장 높은 코사인 유사도 idx: 460
선택된 질문 : 미용실도 잘 골라서 가야 할 것 같더라고요.

선태괸 질문과의 유사도 : 0.363947331905365
원래 문장 : 수학과 특정 학년 관련 공지사항은 어떻게 찾나요?
가장 높은 코사인 유사도 idx: 321
선택된 질문 : 우리 아이는 이 다음엔 그런 단어를 못쓸꺼 같아요

선태괸 질문과의 유사도 : 0.32066261768341064
원래 문장 : 물리학과 공지사항 알림 설정 방법은 무엇인가요?
가장 높은 코사인 유사도 idx: 276
선택된 질문 : 솔드아웃이나 크림 이런 앱 혹시 알아?~

선태괸 질문과의 유사도 : 0.352300226688385
원래 문장 : 화학과 공지사항을 이메일로 받을 수 있나요?
가장 높은 코사인 유사도 idx

100%|█████████▉| 4480/4487 [01:12<00:00, 48.26it/s]

가장 높은 코사인 유사도 idx: 39
선택된 질문 : 취소 하면 되지 않아요?

선태괸 질문과의 유사도 : 0.36812838912010193
원래 문장 : 화학공학과의 세미나 일정 공지사항은 언제인가요?
가장 높은 코사인 유사도 idx: 89
선택된 질문 : 오신다곤 하셨는데 아직 언제 될지 확정이 아니니깐 말씀은 안드렸어

선태괸 질문과의 유사도 : 0.377608060836792
원래 문장 : 산업디자인학과의 워크숍 일정 공지사항은 언제인가요?
가장 높은 코사인 유사도 idx: 89
선택된 질문 : 오신다곤 하셨는데 아직 언제 될지 확정이 아니니깐 말씀은 안드렸어

선태괸 질문과의 유사도 : 0.33559441566467285
원래 문장 : 생명과학과의 학술 대회 일정 공지사항은 언제인가요?
가장 높은 코사인 유사도 idx: 89
선택된 질문 : 오신다곤 하셨는데 아직 언제 될지 확정이 아니니깐 말씀은 안드렸어

선태괸 질문과의 유사도 : 0.3479168713092804
원래 문장 : 수학과의 입학 설명회 공지사항은 언제 열리나요?
가장 높은 코사인 유사도 idx: 89
선택된 질문 : 오신다곤 하셨는데 아직 언제 될지 확정이 아니니깐 말씀은 안드렸어

선태괸 질문과의 유사도 : 0.39752310514450073
원래 문장 : 물리학과의 졸업식 일정 공지사항은 언제인가요?
가장 높은 코사인 유사도 idx: 33
선택된 질문 : 여행가면 숙소는 어디서 예약해요?

선태괸 질문과의 유사도 : 0.34253886342048645
원래 문장 : 화학과의 교내외 수상 내역 공지사항은 무엇이 있나요?
가장 높은 코사인 유사도 idx: 373
선택된 질문 : 아이들 아토피는 없나요?

선태괸 질문과의 유사도 : 0.2958834171295166
원래 문장 : 컴퓨터공학과의 수강신청 관련 공지사항은 무엇인가요?
가장 높은 코사인 유사도 idx: 196
선택된 질문 : 나는 세무사가 되고 싶었는데 한번 찾아봐야겠다.

선태괸 질문과의 유사도 : 0.287569671

100%|██████████| 4487/4487 [01:13<00:00, 61.40it/s]

가장 높은 코사인 유사도 idx: 267
선택된 질문 : 좋아하는 팀 있어?

선태괸 질문과의 유사도 : 0.3427853584289551
원래 문장 : 화학공학과의 학사 일정 변경 공지사항은 어떻게 확인하나요?
가장 높은 코사인 유사도 idx: 39
선택된 질문 : 취소 하면 되지 않아요?

선태괸 질문과의 유사도 : 0.36893320083618164
원래 문장 : 산업디자인학과의 학사 경고 관련 공지사항은 어디서 찾나요?
가장 높은 코사인 유사도 idx: 38
선택된 질문 : 예약을 했는데 숙박업소에서 오류라고 전화 오더라구요

선태괸 질문과의 유사도 : 0.3381373882293701
원래 문장 : 생명과학과의 학적 변경 관련 공지사항은 무엇이 있는지 알려줘
가장 높은 코사인 유사도 idx: 302
선택된 질문 : 아니야 진짜야! 요즘 뜨는애라 말은 못해 신변보호~

선태괸 질문과의 유사도 : 0.3497437834739685


In [27]:
print(f"예측 정확도 : {correct/total_cnt*100} % ")

예측 정확도 : 0.0 % 


In [36]:
f = open("test0723.txt", 'w')
f2 = open("qna.txt", 'r')

test_sentence=f2.readlines()

embedding_vectors=[model.encode(sentence) for sentence in test_sentence]
embedding_vectors=torch.tensor(embedding_vectors)

data2=pd.read_csv("dataset/my_data_0723.csv")
print(embedding_vectors.shape)
    

torch.Size([196, 768])


In [40]:
for i in tqdm(range(len(data))) :
    predict(data.iloc[i]['sentence'])

  1%|          | 46/4487 [00:00<01:11, 61.79it/s]

원래 문장 : 학교 총장님의 주요 업적은 뭐죠?
가장 높은 코사인 유사도 idx: 153
선택된 질문 : 홍익대학교 총장님이 누구야?

선태괸 질문과의 유사도 : 0.4393865168094635


  4%|▎         | 166/4487 [00:02<01:03, 68.34it/s]

원래 문장 : 학교의 발전 목표는 무엇이지?
가장 높은 코사인 유사도 idx: 66
선택된 질문 : 오늘의 학사일정에 대해 알려줘

선태괸 질문과의 유사도 : 0.4498741030693054


  4%|▍         | 194/4487 [00:02<01:02, 68.91it/s]

...

원래 문장 : 생명과학과의 최근 업데이트된 공지사항은 어디서 볼 수 있나요?
가장 높은 코사인 유사도 idx: 186
선택된 질문 : 교수님 연락처 검색하고 싶어

선태괸 질문과의 유사도 : 0.4448527693748474
원래 문장 : 물리학과의 워크숍 일정 공지사항은 언제인가요?
가장 높은 코사인 유사도 idx: 65
선택된 질문 : 12월 학사일정에 대해 알려줘

선태괸 질문과의 유사도 : 0.4369558095932007
원래 문장 : 화학과의 학술 대회 일정 공지사항은 언제인가요?
가장 높은 코사인 유사도 idx: 193
선택된 질문 : 산업공학과 공지사항을 보고싶어

선태괸 질문과의 유사도 : 0.4236736595630646


100%|█████████▉| 4467/4487 [01:07<00:00, 67.11it/s]

원래 문장 : 물리학과의 학사 경고 관련 공지사항은 어디서 찾나요?
가장 높은 코사인 유사도 idx: 193
선택된 질문 : 산업공학과 공지사항을 보고싶어

선태괸 질문과의 유사도 : 0.38302457332611084


100%|██████████| 4487/4487 [01:07<00:00, 66.41it/s]

원래 문장 : 화학공학과의 세미나 일정 공지사항은 언제인가요?
가장 높은 코사인 유사도 idx: 193
선택된 질문 : 산업공학과 공지사항을 보고싶어

선태괸 질문과의 유사도 : 0.44604235887527466
원래 문장 : 산업디자인학과의 워크숍 일정 공지사항은 언제인가요?
가장 높은 코사인 유사도 idx: 195
선택된 질문 : 공지사항 새로 올라온 게 있어?

선태괸 질문과의 유사도 : 0.43905025720596313
원래 문장 : 생명과학과의 학술 대회 일정 공지사항은 언제인가요?
가장 높은 코사인 유사도 idx: 55
선택된 질문 : 2월 학사일정에 대해 알려줘

선태괸 질문과의 유사도 : 0.44360724091529846
